<a id="top"></a>
# Surveying dust structure via GALEX MIS
***

The objective of this tutorial is to create a UV mosaic of a high-latitute cloud, using data from GALEX. We'll query the MAST Archive for Observations, then use the data to create a composite image.

## Learning goals

By the end of this tutorial you will be able to:

- Understand UV images and how they are useful to study dust
- Create a mosaic from several GALEX MIS images
- Learn how to match images using their central coordinates 
- Use masks for circular images

## Introduction

**GALEX background**: The [Galaxy Evolution Explorer (GALEX)](https://archive.stsci.edu/missions-and-data/galex) was a satellite designed to investigate star formation. It observed the sky in two different bands: in the Near UV (NUV) ($1750-27504$ Å) and in the Far UV (FUV) ($1350-1750$ Å). The GALEX database contains over 600 million source measurements in the ultraviolet domain, with some sources having more than one measurement, making them useful for studying their variable sources. 

**GALEX Medium Imaging survey (MIS) background**: Single orbit exposures (1,500s) of 1000 square degrees in positions that match the Sloan Digital Sky Survey (SDSS). These images present a higher resolution in comparison with AIS, since their exposure time was longer.

**Defining some terms:**

- **Intensity map**: image that displays the photons per pixel per second corrected for the relative response of the instrument used.

Clouds can be visible in UV when they are found close to hot stars. The objective of this tutorial is to extract and display an intensity map image of a high-latitude cloud retrieved from GALEX MIS. High-latitude clouds (Galactic latitude: $|b| > 20-30º$) are interesting because they are considered ideal candidates to study triggered star formation.

## Table of Contents
* [Imports](#Imports)
* [MBM 15 Search in GALEX MIS](#MBM-15-Search-in-GALEX-MIS)
* [Product Selection and Download](#Product-Selection-and-Download)
* [Constructing the mosaic](#Constructing-the-mosaic)
    * [Checking orientation](#Checking-orientation)
    * [Saving images to local](#Saving-images-to-local)
    * [Circular mask creation](#Circular-mask-creation)
    * [Expanded image](#Expanded-image)
    * [Limits of image - Coordinates](#Limits-of-image---Coordinates)
    * [Save expanded image to local](#Save-expanded-image-to-local)
    * [Transforming RA/Dec into image coordinates](#Transforming-RA/Dec-into-image-coordinates)
    * [Final mosaic](#Final-mosaic)
* [Exercises](#Exercises)
* [Additional Resources](#Additional-Resources)

## Imports

- `astropy` to use tools needed for performing astronomy and astrophysics with Python, including handling fits files, defining coordinates or choosing the right limits for image visualization.
- `Observations` from _astroquery.mast_ to query the Barbara A. Mikulski Archive for Space Telescopes (MAST).
- `matplotlib` to visualize images.
- `numpy` to manipulate arrays.

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
from astroquery.mast import Observations
from astropy.wcs import WCS
from photutils.aperture import CircularAperture
from reproject import reproject_interp
from reproject.mosaicking import find_optimal_celestial_wcs

## MBM 15 Search in GALEX MIS

We'll access GALEX MIS products using `astroquery.mast`. Let's search for any Observations within a degree of our target, the molecular cloud MBM 15. We should also specify mission with the filter `obs_collection = GALEX`.

Note that we add two additional filters to our query below. First, we use `project="MIS"` to specify the medium imaging survey. There are a few [imaging surveys](http://galex.stsci.edu/doc/CTM/wiki/Public_documentation/Chapter_2.html#GALEX_Science_Surveys) we could choose from, but MIS will give us good coverage of the cloud with a longer exposure time than the all-sky survey. Since [GALEX has two detectors](http://galex.stsci.edu/doc/CTM/wiki/Public_documentation/Chapter_1.html#Detectors), we also specify `filters="FUV"`. Since we're just looking to create a mosaic, it doesn't matter too much which filter we pick; you might have a valid scientific reason to prefer one over the other.

In [ ]:
obs = Observations.query_criteria(objectname="MBM 15", radius="1 deg", obs_collection="GALEX", project="MIS", filters="FUV")
len(obs)

## Product Selection and Download

With nine Observations, we should be able to a mosaic with a good view of the nebula. Now we need to get the data products: the actual files associated with these Observations.

In [ ]:
prod = Observations.get_product_list(obs)
len(prod)

Woah! Over 2000 files from just 9 Observations. Many of these files were intended for calibration or data validation, so we'll just select files that are marked as `"SCIENCE"` type. We'll also keep only files that have been flagged as a "minimum recommended products". These files are considered the absolute minimum needed to do analyze the Observations, which will work well for our mosaic.

In [ ]:
# The manifest will save the download status (i.e. success vs. failed) as well as the file location
manifest = Observations.download_products(prod, productType="SCIENCE", mrp_only = True) 

Great! Now let's open the files into a list of fits objects.

In [ ]:
hdus = [fits.open(filename) for filename in manifest["Local Path"]]

# THOMAS MAYBE PLOT ONE OF THE IMAGES HERE

## Data Wrangling

These fits files that we've downloaded are actually quite large. To keep Notebook runtime short, we're going to scale down the resolution of the image. The "downgrade factor" is, in effect, making the pixels larger and thus have lower memory/processing requirements. With a default downgrade factor of five, our most complicated cell (coming later) will take about 30 seconds to run. To see the original quality image, you can set `downgrade_factor = 1` (i.e. original quality).

In [ ]:
# determine the angular resolution of the image
res = np.abs(hdus[0][0].header['CDELT1'])
# set up a "downgrade factor" to decrease the resolution of the final combined mosaic, because these files are BIG
downgrade_factor = 5     # = 1 for original quality

# use astropy's find_optimal_celestial_wcs to determine the output WCS 
# for the combined image based on the information in the headers
wcs_out, shape_out = find_optimal_celestial_wcs(hdus, resolution=res*downgrade_factor*u.deg)

Since the GALEX data is circular, we need to create a mask for the data. What we want is a set of valid values, surrounded by NaNs. This will allow us to create a mosaic by averaging the valid values; NaNs will be ignored. 

In [ ]:
def galex_mask(map_in, header):
    # define the radius of the circle to be 0.6 degrees, use header information to convert to pixels
    dr = np.rint(0.6 / np.abs(header['CDELT1']))
    app = CircularAperture((header['CRPIX1'], header['CRPIX2']), r=dr)
    
    # convert aperture to mask
    amask = app.to_mask(method='center')
    # pad the mask
    amask = np.pad(np.array(amask.data), 480)
    # convert mask to boolean values
    amask = np.array(amask, dtype=bool)
    
    # mask all pixels outside of aperture to "NaN"
    map_in[~amask] = np.nan
    
    return map_in

In [ ]:
%%time
maps = []
for i, file in enumerate(manifest["Local Path"]):
    with fits.open(file) as ff:
        fmap = ff[0].data
        fhead = ff[0].header
        # for each image, we apply a mask to set all pixels outside of the GALEX footprint to NaN
        fmap = galex_mask(fmap, fhead)
        
        # use reproject to reproject each image to the "combined" output WCS defined earlier
        regmap, foot = reproject_interp((fmap, fhead), wcs_out, shape_out)
        
        # append the reprojected map to a list
        maps.append(regmap)

In [ ]:
# average all of the reprojected maps together
tmap = np.nanmean(np.dstack((maps)), 2)

# ADD AUTOMATIC BRIGHTNESS SCALING

In [ ]:
# plot the new map
fig = plt.figure(0,[9,9])
ax = fig.add_subplot(111, projection=wcs_out)

ax.imshow(tmap, vmin=0.0001, vmax=0.005, origin='lower', cmap='BuPu_r')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
plt.savefig('mbm15_claire.jpg', dpi=600, bbox_inches='tight')

Finally, we have our mosaic! This image corresponds to the 
[Molecular cloud MBM 15](http://simbad.cds.unistra.fr/simbad/sim-id?Ident=mbm+15&submit=submit+id). This molecular cloud belongs to the Orion-Eridanus superbubble, west of the Orion Nebula. More information about this can be found in [Joubaud et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019A%26A...631A..52J/abstract). 

## HERE IS WHERE CLAIRE'S STUFF ENDS :)

## Exercises

Now that we have seen the whole process, let's try to do the same for a different target from the MBM catalog. Let's study now the molecular cloud **TBD**.

Let's start again with the coordinates that SIMBAD provides for the **TBD** (Gal. coordinates, ep = J2000):

- Galactic longitude: **TBD** deg
- Galactic latitude: **TBD** deg

### 1. Coordinate query and data download

Following the steps shown before, perform a query in the GALEX MIS database and download the NUV or FUV intensity maps corresponding to the coordinates provided. Remember to choose a wide radius for the query in order to construct the final mosaic.

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


_**Note**: don't forget to transform the coordinates from the galactic to the icrs frame!_

### 2. Check the orientation of the images

As we did before, check the orientation of the image and make the necessary adjustments if the orientations do not coincide:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


### 3. Save non-central images to local

In order to use the `paste` method, we first need to save the additional images to our local. The central image is the fist one retrieved from the database, so let's save the rest of the images to our local directory in order to use them later. Remember you need to provide the filename which will be used to save the image to your local directory when using `savefig` from `matplotlib`.

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


### 4. Generate the masks for the circular images

Since we are working with circular images, in order to paste them into our extended image we need to only use the part of the image that actually contains information. In order to do this, let's use circular masks as before. Create a circular mask for each of the images retrieved as shown previously.

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


### 5. Create an expanded image from the central one

Using the central image (the first one retrieved from the database), create an extended image by included 0s in the 2D array corresponding to the retrieved intensity map as we did before.

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


### 6. Obtain the limits of the expanded image and save it

Using the technique introduced in Part 1 of this tutorial, and briefly introduced in Part 2 also, obtain the new limits of the extended image we just created using the coordinates of the central image (RA, Dec).

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


### 7. Transform RA, Dec coordinates into image coordinates

Lastly, we need to transform the central coordiantes of the images in order to correctly place them in the mosaic. You can use the function proposed before in this section.

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


### 8. Display the final mosaic

Let's put together everything and display the final mosaic!

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used before here


## Additional Resources

For more information about the MAST archive and details about the tutorial: 

* <a href="https://mast.stsci.edu/api/v0/index.html">MAST API</a> <br>
* <a href="https://galex.stsci.edu/GR6/">GALEX via MAST</a> <br>

## About this Notebook

**Author**: Clara Puerto Sánchez, Claire Murray <br>
**Keyword(s)**: Tutorial, mosaic, dust <br>
**Last Updated**: Apr 2023 <br>

For support, please contact the Archive HelpDesk at archive@stsci.edu.
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>